In [44]:
import pandas as pd
import numpy as np
import json
import os
import joblib
import ast
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# --- Configuration for Local Environment ---
DATA_DIR = 'data'
MODELS_DIR = 'saved_models'
RESULTS_DIR = 'results'

# Set to True if you want to rebuild all models from scratch
FORCE_RETRAIN = False

# Minimum ratings required to build a model for a critic
MIN_RATINGS_PER_CRITIC = 5

In [45]:
def parse_tags(data_str):
    """
    Safely parses a string that could be a single or double-encoded list of tags.
    """
    if not isinstance(data_str, str):
        return ''
    try:
        # First, try to evaluate the string as a Python literal.
        parsed_data = ast.literal_eval(data_str)

        # If the result of the first parse is *still* a string, it's likely
        # double-encoded, so we parse it again using json.loads.
        if isinstance(parsed_data, str):
            final_list = json.loads(parsed_data)
        else:
            final_list = parsed_data

        # Final check to ensure we have a list before joining.
        if isinstance(final_list, list):
            return ' '.join(final_list)
        else:
            return ''
    except (ValueError, SyntaxError, json.JSONDecodeError, TypeError):
        # If any parsing step fails, return an empty string.
        return ''

def preprocess_data(games_df, ratings_df):
    """Merges, cleans, and prepares the data for modeling."""
    ratings_df = ratings_df.rename(columns={'id': 'rating_id'})
    if 'appid' in games_df.columns:
        games_df = games_df.drop(columns=['appid'])

    merged_df = pd.merge(ratings_df, games_df, left_on='game_id', right_on='id', how='left')

    merged_df['will_skip'] = merged_df['score'].isnull() | (merged_df['score'] == 'skipped')
    merged_df['score_numeric'] = pd.to_numeric(merged_df['score'], errors='coerce')

    text_cols = ['user_tags', 'developer_genres', 'developers', 'publishers']
    for col in text_cols:
        merged_df[col] = merged_df[col].fillna('')

    if 'user_tags' in merged_df.columns:
         merged_df['user_tags'] = merged_df['user_tags'].apply(parse_tags) # Use new parser

    for col in ['developer_genres', 'developers', 'publishers']:
        if col in merged_df:
             merged_df[col] = merged_df[col].str.replace(',', ' ')

    merged_df['metacritic_score'] = pd.to_numeric(merged_df['metacritic_score'], errors='coerce')
    merged_df['price_usd'] = pd.to_numeric(merged_df['price_usd'], errors='coerce')
    merged_df['release_year'] = pd.to_datetime(merged_df['release_date'], errors='coerce', format='mixed').dt.year

    return merged_df

def build_feature_pipeline():
    """Builds a scikit-learn pipeline to transform raw data into model-ready features."""
    numeric_features = ['metacritic_score', 'price_usd', 'release_year']
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

    text_transformer = TfidfVectorizer(stop_words='english', max_features=100, min_df=1)

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('tags', text_transformer, 'user_tags'),
            ('genres', TfidfVectorizer(max_features=50, min_df=1), 'developer_genres'),
            ('devs', TfidfVectorizer(max_features=50, min_df=1), 'developers'),
            ('pubs', TfidfVectorizer(max_features=50, min_df=1), 'publishers')
        ],
        remainder='drop'
    )
    return preprocessor

In [46]:
def run_process(data_dir, models_dir, results_dir, force_retrain):
    """Main function to run the entire analysis pipeline."""
    print("Loading data...")
    games_path = os.path.join(data_dir, 'games_details.csv')
    ratings_path = os.path.join(data_dir, 'ratings.csv')
    try:
        games_df = pd.read_csv(games_path)
        ratings_df = pd.read_csv(ratings_path)
    except FileNotFoundError as e:
        print(f"Error: {e}. Make sure your CSV files are in the '{data_dir}' directory.")
        return

    os.makedirs(models_dir, exist_ok=True)
    os.makedirs(results_dir, exist_ok=True)

    print("Preprocessing data...")
    data = preprocess_data(games_df.copy(), ratings_df.copy())

    all_games_features = games_df.drop_duplicates(subset=['id']).copy()
    if 'appid' in all_games_features.columns:
        all_games_features = all_games_features.drop(columns=['appid'])

    text_cols = ['user_tags', 'developer_genres', 'developers', 'publishers']
    for col in text_cols:
        all_games_features[col] = all_games_features[col].fillna('')
    if 'user_tags' in all_games_features.columns:
        all_games_features['user_tags'] = all_games_features['user_tags'].apply(parse_tags)
    for col in ['developer_genres', 'developers', 'publishers']:
        if col in all_games_features:
             all_games_features[col] = all_games_features[col].str.replace(',', ' ')

    all_games_features['metacritic_score'] = pd.to_numeric(all_games_features['metacritic_score'], errors='coerce')
    all_games_features['price_usd'] = pd.to_numeric(all_games_features['price_usd'], errors='coerce')
    all_games_features['release_year'] = pd.to_datetime(all_games_features['release_date'], errors='coerce', format='mixed').dt.year

    print("Fitting the feature preprocessor on all games...")
    preprocessor = build_feature_pipeline()
    preprocessor.fit(all_games_features)
    print("Preprocessor fitted.")

    # --- NEW: Get feature names from the fitted preprocessor ---
    try:
        feature_names = preprocessor.get_feature_names_out()
    except Exception as e:
        print(f"Warning: Could not get feature names from the preprocessor. Error: {e}")
        feature_names = None

    critics = data['critic_id'].unique()
    all_predictions = []
    all_importances = [] # --- NEW: List to store importance results ---

    print(f"\nFound {len(critics)} critics. Starting model processing loop...")
    for critic_id in critics:
        critic_data = data[data['critic_id'] == critic_id]

        if len(critic_data) < MIN_RATINGS_PER_CRITIC:
            print(f"Skipping critic {critic_id}: not enough ratings.")
            continue

        clf_path = os.path.join(models_dir, f'{critic_id}_classifier.joblib')
        reg_path = os.path.join(models_dir, f'{critic_id}_regressor.joblib')

        print(f"--- Training models for critic: {critic_id} ---")

        X_class_transformed = preprocessor.transform(critic_data)
        y_class = critic_data['will_skip']
        clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
        clf.fit(X_class_transformed, y_class)
        joblib.dump(clf, clf_path)

        rated_games = critic_data.dropna(subset=['score_numeric'])
        reg = None
        if len(rated_games) >= MIN_RATINGS_PER_CRITIC / 2:
            X_reg_transformed = preprocessor.transform(rated_games)
            y_reg = rated_games['score_numeric']
            reg = RandomForestRegressor(n_estimators=100, random_state=42)
            reg.fit(X_reg_transformed, y_reg)
            joblib.dump(reg, reg_path)

        # --- NEW: Extract and store top 5 feature importances ---
        if feature_names is not None:
            # Get importances for the skip model (classifier)
            skip_importances = pd.DataFrame({
                'feature': feature_names,
                'importance': clf.feature_importances_
            }).sort_values(by='importance', ascending=False).head(5)
            skip_importances['critic_id'] = critic_id
            skip_importances['model_type'] = 'skip_prediction'
            all_importances.append(skip_importances)

            # Get importances for the score model (regressor), if it was trained
            if reg:
                score_importances = pd.DataFrame({
                    'feature': feature_names,
                    'importance': reg.feature_importances_
                }).sort_values(by='importance', ascending=False).head(5)
                score_importances['critic_id'] = critic_id
                score_importances['model_type'] = 'score_prediction'
                all_importances.append(score_importances)

        # --- Generate Predictions for ALL games ---
        X_all_games_transformed = preprocessor.transform(all_games_features)
        skip_probs = clf.predict_proba(X_all_games_transformed)[:, 1]
        pred_scores = reg.predict(X_all_games_transformed) if reg else np.nan

        predictions_df = all_games_features[['id', 'name']].copy()
        predictions_df['critic_id'] = critic_id
        predictions_df['predicted_skip_probability'] = skip_probs
        predictions_df['predicted_score'] = pred_scores
        all_predictions.append(predictions_df)

    # --- Save Prediction Results ---
    if all_predictions:
        final_pred_df = pd.concat(all_predictions, ignore_index=True)
        pred_output_path = os.path.join(results_dir, 'critic_predictions.csv')
        final_pred_df.to_csv(pred_output_path, index=False)
        print(f"\n✅ Predictions saved to '{pred_output_path}'")
    else:
        print("\nNo predictions were generated.")

    # --- NEW: Save Feature Importances to a CSV ---
    if all_importances:
        final_imp_df = pd.concat(all_importances, ignore_index=True)
        imp_output_path = os.path.join(results_dir, 'critic_feature_importances.csv')
        final_imp_df.to_csv(imp_output_path, index=False)
        print(f"✅ Feature importances saved to '{imp_output_path}'")

In [47]:
run_process(
    data_dir=DATA_DIR,
    models_dir=MODELS_DIR,
    results_dir=RESULTS_DIR,
    force_retrain=FORCE_RETRAIN
)

Loading data...
Preprocessing data...
Fitting the feature preprocessor on all games...
Preprocessor fitted.

Found 10 critics. Starting model processing loop...
--- Training models for critic: 1 ---
--- Training models for critic: 2 ---
--- Training models for critic: 3 ---
--- Training models for critic: 4 ---
--- Training models for critic: 5 ---
--- Training models for critic: 6 ---
--- Training models for critic: 7 ---
--- Training models for critic: 8 ---
--- Training models for critic: 10 ---
--- Training models for critic: 9 ---

✅ Predictions saved to 'results/critic_predictions.csv'
✅ Feature importances saved to 'results/critic_feature_importances.csv'


In [48]:
# Zip the saved_models directory
!zip -r /content/saved_models.zip /content/saved_models

# Download the zip file
from google.colab import files
files.download('/content/saved_models.zip')

  adding: content/saved_models/ (stored 0%)
  adding: content/saved_models/7_classifier.joblib (deflated 87%)
  adding: content/saved_models/3_classifier.joblib (deflated 87%)
  adding: content/saved_models/5_classifier.joblib (deflated 87%)
  adding: content/saved_models/10_regressor.joblib (deflated 92%)
  adding: content/saved_models/4_classifier.joblib (deflated 87%)
  adding: content/saved_models/2_regressor.joblib (deflated 89%)
  adding: content/saved_models/2_classifier.joblib (deflated 89%)
  adding: content/saved_models/1_regressor.joblib (deflated 90%)
  adding: content/saved_models/8_regressor.joblib (deflated 89%)
  adding: content/saved_models/5_regressor.joblib (deflated 91%)
  adding: content/saved_models/6_classifier.joblib (deflated 93%)
  adding: content/saved_models/6_regressor.joblib (deflated 87%)
  adding: content/saved_models/9_classifier.joblib (deflated 93%)
  adding: content/saved_models/8_classifier.joblib (deflated 87%)
  adding: content/saved_models/7_regr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>